<a href="https://colab.research.google.com/github/jokefun022/Google-Drive/blob/main/24_08_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 6.9 MB/s eta 0:00:00


In [ ]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

In [ ]:
%pip install keras-preprocessing

  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl.metadata (1.9 kB)
Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)


In [ ]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints

class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 2.0.6
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = K.squeeze(K.dot(x, K.expand_dims(self.W)), axis=-1)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)

        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [ ]:
import emoji
import gensim
import numpy as np
import pandas as pd
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from nltk import TweetTokenizer
from numpy import asarray, zeros


def ReadOpen(filename, Labelfile):
	data = []
	tokenizer_tweet = TweetTokenizer()

	with open(filename, 'r', encoding="utf-8", errors="replace") as readFile:
		lines = readFile.readlines()

	for line in lines:
		temp = []
		sentence = ' '.join(line.strip().split(','))
		for token in tokenizer_tweet.tokenize(sentence):
			temp.append(token.lower())
		data.append(temp)

	labels_pd = pd.read_csv(Labelfile, index_col=False, header=None)
	labels = labels_pd.values.squeeze()

	return data, labels, len(lines)


def AverageVectorPerTweet(data, model_word2vec):
    avg = []
    for i in range(len(data)):
        row = []
        for j in data[i]:
            if j in model_word2vec.vocab:
                row.append(model_word2vec[j])
        if row:
            row = np.asarray(row)
            avg.append((np.average(row, axis=0)).tolist())
        else:
            avg.append(np.zeros((200,)).tolist())
    return avg


def AverageVectorPerEmoji(data, model_emoji2vec):
    avg = []
    for i in range(len(data)):
        row = []
        for j in data[i]:
            if j in model_emoji2vec.vocab:
                row.append(model_emoji2vec[j])
        if row:
            row = np.asarray(row)
            avg.append((np.average(row, axis=0)).tolist())
        else:
            avg.append(np.zeros((200,)).tolist())
    return avg


def ml_read_data(data_file, label_file, glove_model, emoji2vec_model):
    data, label, count = ReadOpen(data_file, label_file)

    embedded_sentences = AverageVectorPerTweet(data, glove_model)
    embedded_sentences_emoji = AverageVectorPerEmoji(data, emoji2vec_model)
    embedded_sentences_emoji = np.concatenate((np.array(embedded_sentences), np.array(embedded_sentences_emoji)),
                                              axis=1).tolist()
    X = np.array(embedded_sentences)

    X = np.array(X.tolist())
    y = np.array(label)
    indices = np.random.permutation(len(X))

    X = X[indices]
    y = y[indices]

    X_emoji = np.array(embedded_sentences_emoji)

    X_emoji = np.array(X_emoji.tolist())
    y_emoji = np.array(label)

    X_emoji = X_emoji[indices]
    y_emoji = y_emoji[indices]
    return X, y, X_emoji, y_emoji


def Preprocess(docs, count, glove_model, emoji2vec_model, get_emoji2vec=True):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(docs)
    encoded_docs = tokenizer.texts_to_sequences(docs)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    maxlen = len(padded_docs[0])
    nf = 0
    embedding_matrix = zeros((count, 200))
    for word, i in tokenizer.word_index.items():
        if word in glove_model.vocab:
            embedding_matrix[i] = glove_model[word]
        else:
            new_em = []
            em = [item['emoji'] for item in emoji.emoji_list(word)]
            for ej in em:
                for c in ej:
                    if emoji.is_emoji(c):
                        new_em.append(c)
            try:
                if new_em:
                    row = []
                    for e in new_em:
                        row.append(emoji2vec_model[e])
                    if get_emoji2vec:
                        embedding_matrix[i] = np.average(np.asarray(row), axis=0).tolist()
                    else:
                        embedding_matrix[i] = [0] * 200
                else:
                    embedding_matrix[i] = [0] * 200
            except:
                embedding_matrix[i] = [0] * 200
                nf += 1

    return padded_docs, embedding_matrix, maxlen, tokenizer


def preprocess_test(tokenizer, maxlen, test_docs):
    test_encoded_docs = tokenizer.texts_to_sequences(test_docs)
    test_padded_docs = pad_sequences(test_encoded_docs, maxlen=maxlen, padding='post')
    return test_padded_docs

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Activation, Bidirectional, Dense, Flatten, Embedding
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers.core import *
from tensorflow.keras.optimizers import Adam
# from attention_layer import Attention # Removed this line


def PrepModel(count, embedding_matrix, l, lrate=0.001):
    model = Sequential()
    e = Embedding(count, 200, weights=[embedding_matrix], input_length=l, trainable=False)
    model.add(e)
    model.add(Dropout(0.25))
    model.add(Bidirectional(
        LSTM(256, kernel_initializer='he_normal', recurrent_activation='sigmoid', return_sequences=True,
             activation='tanh')))
    model.add(Dropout(0.4))
    model.add(Bidirectional(
        LSTM(256, kernel_initializer='he_normal', recurrent_activation='sigmoid', return_sequences=True,
             activation='tanh')))
    model.add(Dropout(0.4))
    model.add(Attention())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=lrate), loss='binary_crossentropy', metrics=['acc'])
    return model

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from gensim.models import KeyedVectors
import joblib
# from data_utils import ml_read_data # Removed this line